In [ ]:
import os
import json

kaggle_info = {"username":"aadityatiwary2004","key":"41bd25eaf1db3bc535973f652664"}

# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Write the kaggle.json file
with open(os.path.expanduser('~/.kaggle/kaggle.json'), 'w') as f:
    json.dump(kaggle_info, f)

# Set permissions for the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ismailpromus/skin-diseases-image-dataset

In [ ]:
!unzip skin-diseases-image-dataset.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# === Dataset path ===
data_dir = r"Skin_Diseases" 

In [ ]:
# === Data generators with augmentation and split ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% val
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

In [ ]:
# === Load MobileNetV2 as base model ===
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Stage 1: freeze base layers

In [ ]:
# === Add custom layers ===
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x) # Increased units
x = Dropout(0.4)(x) # Increased dropout
x = Dense(128, activation='relu')(x) # Added dense layer
x = Dropout(0.3)(x) # Added dropout
x = Dense(64, activation='relu')(x) # Added another dense layer
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# === Compile and train (Stage 1: feature extraction) ===
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history1 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10
)

In [ ]:
# === Stage 2: Fine-tuning last 50 layers of MobileNetV2 ===
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history2 = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=25
)


In [ ]:
# === Evaluate the model on the validation set ===
loss, accuracy = model.evaluate(val_generator, steps=len(val_generator))
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
model.save("skin_disease_model.h5")